In [1]:
import nltk
from nltk.util import bigrams
from nltk.util import pad_sequence
from nltk.lm.preprocessing import pad_both_ends
from nltk.util import everygrams
from nltk.lm.preprocessing import flatten
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
import sklearn
from nltk.classify import scikitlearn
import csv
import pandas as pd
from nltk import word_tokenize
nltk.download('punkt')
from nltk.lm import Laplace
import re
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor, MultiOutputClassifier
from scipy.sparse import hstack
from sklearn.metrics import accuracy_score, precision_score
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix, classification_report
import matplotlib as plt
import statistics
from sklearn import preprocessing
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor, MultiOutputClassifier
from scipy.sparse import hstack
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import multilabel_confusion_matrix, classification_report
import matplotlib as plt
import statistics
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.metrics import accuracy_score, precision_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\brown\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brown\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def read_and_preprocess_dataset(file, labels = 0):
    df = pd.read_csv(file, sep='\t')

    if labels != 0:
        theme_df = pd.read_csv(labels, sep='\t')
        theme_df = theme_df.iloc[:, 1:]
    
    le = preprocessing.LabelEncoder()
    prep_df = df.copy()
    le.fit(prep_df['Stance'])
    prep_df['Stance'] = le.transform(prep_df['Stance'])
    
    # Replace Punctuation
    prep_df['Conclusion'] = prep_df['Conclusion'].str.replace(r'[^\w\s]+', '', regex=True)
    prep_df['Premise']    = prep_df['Premise'].str.replace(r'[^\w\s]+', '', regex=True)
    
    #Lowercase 'Conclusion' and 'Premise' column on prep_df 
    prep_df.loc[:, 'Conclusion'] = prep_df.Conclusion.str.lower()
    prep_df.loc[:, 'Premise'] = prep_df.Premise.str.lower()
    
    # Remove Stopwords
    for idx,row in prep_df.iterrows():
        copyConc = prep_df.loc[idx,'Conclusion'].split(' ')
        copyPremise = prep_df.loc[idx,'Premise'].split(' ')
        newConc = ''
        newPrem = ''
    
        for word in copyConc:
            if word not in stopwords.words('english'):
                newConc = newConc+' '+word
            prep_df.loc[idx,'Conclusion'] = newConc
        
        for word in copyPremise:
            if word not in stopwords.words('english'):
                newPrem = newPrem+' '+word
            prep_df.loc[idx,'Premise'] = newPrem
    if labels != 0:
        return prep_df, theme_df
    else:
        return prep_df

In [11]:
### Creating a random forest model for each feature to train on
def ensemble_train(concs, prems, train_set, train_labels):
    
    rf_model = RandomForestClassifier()
    rf_multi1 = MultiOutputClassifier(rf_model)
    rf_multi2 = MultiOutputClassifier(rf_model)
    rf_multi3 = MultiOutputClassifier(rf_model)

    rf_model_fit1 = rf_multi1.fit(concs, 
                            train_labels)

    rf_model_fit2 = rf_multi2.fit(prems, 
                            train_labels)

    rf_model_fit3 = rf_multi3.fit(train_set['Stance'].values.reshape(-1,1), 
                            train_labels)
    
    return rf_model_fit1, rf_model_fit2, rf_model_fit3

In [44]:
### Predicting each individual feature using one model, then using maximum voting for each category, 
### and compiling it together for the final category.

def ensemble_validation(model1, model2, model3, val_df, val_labels, 
                        val_concs, val_prems):
    pred_Conc_Targets   = model1.predict(val_concs)
    pred_Prem_Targets   = model2.predict(val_prems)
    pred_Stance_Targets = model3.predict(val_df['Stance'].values.reshape(-1,1))

    final_preds = []

    for i in range(len(pred_Conc_Targets)):
        pred_list = []
        for j in range(len(pred_Conc_Targets[i])):
            pred_values = [pred_Conc_Targets[i][j], pred_Prem_Targets[i][j], pred_Stance_Targets[i][j]]
            mode = statistics.mode(pred_values)
            pred_list.append(mode)
        final_preds.append(pred_list)
        
    cm = classification_report(val_labels, final_preds)
    print(cm)
    return final_preds

In [34]:
def ensemble_test(model1, model2, model3, val_df, 
                        val_concs, val_prems):
    pred_Conc_Targets   = model1.predict(val_concs)
    pred_Prem_Targets   = model2.predict(val_prems)
    pred_Stance_Targets = model3.predict(val_df['Stance'].values.reshape(-1,1))

    final_preds = []

    for i in range(len(pred_Conc_Targets)):
        pred_list = []
        for j in range(len(pred_Conc_Targets[i])):
            pred_values = [pred_Conc_Targets[i][j], pred_Prem_Targets[i][j], pred_Stance_Targets[i][j]]
            mode = statistics.mode(pred_values)
            pred_list.append(mode)
        final_preds.append(pred_list)
        
    return final_preds

In [54]:
train_df, label_df       = read_and_preprocess_dataset('arguments-training.tsv', 'labels-training.tsv')
valid_df,  validlabel_df = read_and_preprocess_dataset('arguments-validation.tsv', 'labels-validation.tsv')
test_df                  = read_and_preprocess_dataset('arguments-test.tsv')

In [14]:
tfidf = TfidfVectorizer()
train_Concs        = tfidf.fit_transform(train_df['Conclusion'])
valid_Concs        = tfidf.transform(valid_df['Conclusion'])
test_Concs        = tfidf.transform(test_df['Conclusion'])

train_Prems        = tfidf.fit_transform(train_df['Premise'])
valid_Prems        = tfidf.transform(valid_df['Premise'])
test_Prems        = tfidf.transform(test_df['Premise'])

In [15]:
model_1, model_2, model_3 = ensemble_train(train_Concs, train_Prems,
                                           train_df, label_df)

In [45]:
final_val_predictions = ensemble_validation(model_1, model_2, model_3,
                                           valid_df, validlabel_df,
                                            valid_Concs, valid_Prems)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       251
           1       0.67      0.00      0.01       496
           2       0.00      0.00      0.00       138
           3       0.00      0.00      0.00       103
           4       0.75      0.12      0.21       575
           5       0.00      0.00      0.00       164
           6       0.00      0.00      0.00       132
           7       0.00      0.00      0.00       130
           8       0.71      0.10      0.17       759
           9       0.60      0.04      0.08       488
          10       0.00      0.00      0.00       172
          11       0.00      0.00      0.00       455
          12       0.00      0.00      0.00        60
          13       0.00      0.00      0.00       127
          14       0.00      0.00      0.00       633
          15       0.00      0.00      0.00       268
          16       0.57      0.05      0.09       687
          17       1.00    

C:\Users\brown\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\brown\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [70]:
final_train_predictions = ensemble_validation(model_1, model_2, model_3,
                                           train_df, label_df,
                                            train_Concs, train_Prems)

              precision    recall  f1-score   support

           0       0.88      0.27      0.41       988
           1       0.94      0.17      0.28      1395
           2       0.00      0.00      0.00       247
           3       0.00      0.00      0.00       172
           4       0.87      0.31      0.46      1512
           5       1.00      0.02      0.04       610
           6       0.83      0.25      0.39       625
           7       1.00      0.01      0.02       382
           8       0.89      0.50      0.64      2000
           9       0.87      0.53      0.66      1728
          10       0.95      0.13      0.23       568
          11       0.92      0.07      0.13      1177
          12       0.00      0.00      0.00       207
          13       1.00      0.02      0.03       395
          14       1.00      0.03      0.06      1332
          15       1.00      0.03      0.06       806
          16       0.88      0.50      0.64      2081
          17       0.96    

C:\Users\brown\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\brown\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\brown\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [72]:
final_test_predictions = ensemble_test(model_1, model_2, model_3,
                                           test_df,
                                            test_Concs, test_Prems)

In [68]:
Class_list = list(label_df.columns.values)

class_dict = {}
n = 0
for category in Class_list:
    class_dict[n] = category
    n += 1
print(len(class_dict.values()))
print(class_dict.values())

20
dict_values(['Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity'])


In [69]:
val_df_results = pd.DataFrame.from_records(final_val_predictions)
val_df_results.columns = class_dict.values()
val_df_results.head(20)

,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,Security: societal,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [71]:
train_df_results = pd.DataFrame.from_records(final_train_predictions)
train_df_results.columns = class_dict.values()
train_df_results.head(20)

,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,Security: societal,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
6,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0


In [73]:
test_df_results = pd.DataFrame.from_records(final_test_predictions)
test_df_results.columns = class_dict.values()
test_df_results.head(20)

,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,Security: societal,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
## Val results
val_df_results.to_csv('val_df_results.tsv', sep = '\t')

## train results
train_df_results.to_csv('train_df_results.tsv', sep = '\t')

## test results
test_df_results.to_csv('test_df_results.tsv', sep = '\t')

In [ ]:
val